# Tableau Dashboard Data for Retail WoW Leaderboard Stats

- Goal is to build a Tableau dashboard for looking at achievement leaderboard stats (and leaderboard guildmates)
- Original data is from the Subscript web app (in the insight_data_science repo)
- New data includes detailed profile information including class and active spec

In [19]:
import os
import pandas as pd
import google_sheets as gs
import wow_retail_api as wow
import configparser as cp
import json
import file_concatenator as fc
import requests

In [27]:
def get_access_token (blizzard_key, blizzard_secret):
    """Get access token to access the blizzard API"""
    r = requests.post('https://us.battle.net/oauth/token', data={'grant_type': 'client_credentials'},
                  auth=(blizzard_key, blizzard_secret))
    unpacked = json.loads(r.text)
    access_token = unpacked['access_token']
    return access_token

In [29]:
def get_wow_profile (realm, player, token):
    """Retrievesthe public profile for a player using the Blizzard API"""
    url = 'https://us.api.blizzard.com/profile/wow/character/' + realm \
          + '/' + player + '?namespace=profile-us&locale=en_US&access_token=' + token
    try:
        r = requests.get(url)
    except:
        pass
    unpacked = json.loads(r.text)
    try:
        row = dict(id = unpacked['id'], name = unpacked['name'], gender = unpacked['gender']['name'],
          faction = unpacked['faction']['name'], race = unpacked['race']['name'],
          character_class = unpacked['character_class']['name'],
          active_spec = unpacked['active_spec']['name'], realm = unpacked['realm']['slug'],
          guild = unpacked['guild']['name'], level = unpacked['level'],
          achievement_points = unpacked['achievement_points'],
          last_login = unpacked['last_login_timestamp'],
          average_item_level = unpacked['average_item_level'],
          equipped_item_level = unpacked['equipped_item_level'])
        return row
    except:
        pass

### Get API credentials

In [30]:
f_config = '../wow_config.ini'
config = cp.ConfigParser()
config.read(f_config)
key = config.get('client', 'blizzard')
secret = config.get('secret', 'blizzard')
token = get_access_token (key, secret)

### Get Character List
- Recycling the leaderboards and their guildmates from the Subscript project

In [12]:
df = pd.read_csv(os.path.join('data', 'final_time_stats.csv'))
df = df.dropna()
unnamed = [c for c in df.columns.values if 'unnamed' in c.lower()]
df = df.drop(unnamed, axis = 1)
df

,player,realm,gear_score,last_login,time_since_login,2014-01,2015-01,2016-01,2017-01,2018-01,...,2020-11,2014-12,2015-12,2016-12,2017-12,2018-12,2019-12,2020-12,engagement,status
0,runeknight,whisperwind,437.0,2020-05-31,5 days 00:00:00.000000000,0.0,0.0,4.0,0.0,5.0,...,0.0,7.0,7.0,0.0,1.0,2.0,3.0,0.0,0.0,active
1,chrichton,whisperwind,451.0,2020-06-04,1 days 00:00:00.000000000,0.0,4.0,1.0,1.0,6.0,...,0.0,23.0,6.0,7.0,2.0,1.0,3.0,0.0,0.0,active
2,shìfted,whisperwind,447.0,2020-06-07,-2 days +00:00:00.000000000,0.0,10.0,1.0,0.0,6.0,...,0.0,53.0,16.0,1.0,0.0,3.0,48.0,0.0,0.0,active
3,láwdamercy,whisperwind,329.0,2020-06-07,-2 days +00:00:00.000000000,0.0,7.0,0.0,0.0,10.0,...,0.0,21.0,10.0,2.0,0.0,2.0,10.0,0.0,0.0,active
4,portsforsale,whisperwind,268.0,2019-10-23,226 days 00:00:00.000000000,3.0,10.0,2.0,6.0,1.0,...,0.0,19.0,4.0,7.0,0.0,4.0,1.0,0.0,1.0,risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169449,xanarian,arathor,416.0,2019-09-25,254 days 00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,0.0,2.0,0.0,3.0,4.0,6.0,35.0,0.0,2.0,lapsed
169450,trails,arathor,435.0,2019-10-17,232 days 00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,0.0,2.0,0.0,3.0,4.0,6.0,35.0,0.0,1.0,risk
169451,teyvarix,drenden,476.0,2020-06-10,-5 days +00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,0.0,2.0,0.0,3.0,4.0,3.0,16.0,0.0,0.0,active
169452,daxus,drenden,465.0,2020-06-10,-5 days +00:00:00.000000000,2.0,0.0,1.0,5.0,0.0,...,0.0,2.0,0.0,3.0,4.0,3.0,13.0,0.0,0.0,active


### Get character basics

In [33]:
df_profile = pd.DataFrame()
count = 0

# The loop is only 5 rows for demo purposes
for index, row in df.iloc[:5].iterrows():
    try:
        row_profile = get_wow_profile(row.realm, row.player, token)
        df_profile = df_profile.append(row_profile, ignore_index = True)  
        if count % 1000 == 0:
            df_profile.to_csv(os.path.join('data','profile','profile' + '_'+ str(count) + '.csv'))
            df_profile = pd.DataFrame()
        count += 1
        print(count, end = ' ')
    except:
        pass
    
df_profile

1 2 3 4 5 

,achievement_points,active_spec,average_item_level,character_class,equipped_item_level,faction,gender,guild,id,last_login,level,name,race,realm
0,10665.0,Beast Mastery,452.0,Hunter,452.0,Horde,Male,Scotch and Soda,108852375.0,1.599147e+12,120.0,Chrichton,Vulpera,whisperwind
1,10665.0,Restoration,451.0,Druid,450.0,Horde,Male,Scotch and Soda,109071045.0,1.596777e+12,120.0,Shìfted,Highmountain Tauren,whisperwind
2,10665.0,Discipline,329.0,Priest,329.0,Horde,Male,Scotch and Soda,109133210.0,1.593437e+12,120.0,Láwdamercy,Tauren,whisperwind
3,12985.0,Fire,269.0,Mage,268.0,Horde,Female,Scotch and Soda,109579770.0,1.571866e+12,120.0,Portsforsale,Goblin,whisperwind


### Concatenate Files

In [23]:
df_profile = fc.concat(os.path.join('data','profile'))

### Upload to Google Sheets

In [20]:
# Connect to google drive
service = gs.google_connect()

In [24]:
# Make a new sheet for the profile data
sheet_id = gs.create_sheets (service, 'wow_profiles')

In [25]:
# Write detailed profile data to the google sheet
gs.write_sheet(service, sheet_id, df_profile, 'Sheet1!A:ZZZ')

Sheet successfully Updated


In [14]:
# Write the original achievement stats file to google sheets
sheet_id = gs.create_sheets (service, 'wow_profile_achieves')
gs.write_sheet(service, sheet_id, df.iloc[:1700,:], 'Sheet1!A:ZZZ')

Sheet successfully Updated
